In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 1
setup_one_gpu(GPU)

Picking GPU 1


In [2]:
import numpy as np
import os.path as osp
from geo_tool import Point_Cloud
from tf_lab.nips.helper import pclouds_centered_and_half_sphere, center_pclouds_in_unit_sphere
from tf_lab.iclr.helper import load_multiple_version_of_pcs
from tf_lab.data_sets.shape_net import snc_category_to_synth_id
from tf_lab.evaluate.generative_pc_nets import entropy_of_occupancy_grid, \
jensen_shannon_divergence, minimum_mathing_distance, coverage
from tf_lab.iclr.evaluator import Evaluator

In [3]:
%load_ext autoreload
%matplotlib inline
%autoreload 2

In [7]:
# Parameters
n_pc_samples = 2048
mmd_loss = 'emd'
skip = ['train', 'test' ]

do_cov = True
do_mmd = True
do_jsd = False
sample_metrics = False
pc_normalizer = center_pclouds_in_unit_sphere

In [11]:
top_in_data = '/orions4-zfs/projects/optas/DATA/OUT/iclr/synthetic_samples/mit_wu_isosurface_extracted/'
all_classes = ['car', 'table', 'sofa', 'rifle']
boost_samples = 3

In [12]:
all_classes = ['rifle']
for class_name in all_classes:
    print class_name
    
    jsd_scores = []
    mmd_scores = []
    cov_scores = []

    evaluator = Evaluator(class_name, norm=pc_normalizer)
    evaluator.load_gt_data()
    
    if do_jsd:
        evaluator.prepare_gt_grid_variables()
    
    sample_file = osp.join(top_in_data, class_name + '.npz')
    
    evaluator.prepare_sample_data(sample_file, boost_sample=boost_samples)
    
    if do_jsd:
        evaluator.prepare_sample_data(sample_file, boost_sample=1)
        jsd_scores.append(evaluator.compute_jsd(skip=skip))

    if do_mmd:
        score = evaluator.compute_mmd(loss=mmd_loss, sample_estimator=sample_metrics, skip=skip)
        mmd_scores.append(np.mean(score['val']))
        print mmd_scores[-1]

    if do_cov:
        score = evaluator.compute_coverage(loss=mmd_loss, sample_estimator=sample_metrics, skip=skip)
        cov_scores.append(score['val'])
        print cov_scores[-1]
    print np.mean(cov_scores), np.mean(mmd_scores)
    print

rifle
val 106.455 30.2683
106.455
val 0.703389830508 0.0
0.703389830508
0.703389830508 106.455



0.570621468927 0.000896995


In [ ]:
# On the validation set: 1K is about 3x it's size (not true for all classes...)
# So this makes it fair when we used in
# other methods 3x boost. Also Wu-et.al has seen the test/val data which makes it better for them. 
# Thus i suggest to report the average here too.

In [ ]:
# Load ALL Wu Data, compute grid variable.
wu_data = np.load(osp.join(top_in_data, class_name + '.npz'))
wu_data = pc_normalizer(wu_data[wu_data.keys()[0]])
assert(wu_data.shape[1] == n_pc_samples)

if do_jsd:
    _, wu_grid_var = entropy_of_occupancy_grid(wu_data, voxel_resolution, in_sphere=cmp_in_sphere)
    if use_uniform_prior:
        wu_grid_var += 1

# Load Ground-Truth Train Data (All data of the class.)
top_gt_dir = '/orions4-zfs/projects/optas/DATA/OUT/iclr/evaluations/gt_data/'

all_class_data = []
for split in ['train', 'test', 'val']:
    gt_file = osp.join(top_gt_dir, class_name + '_' + split + '.npz')
    gt_data = np.load(gt_file)
    gt_data = pc_normalizer(gt_data[gt_data.keys()[0]])
    all_class_data.append(gt_data)

all_class_data = np.vstack(all_class_data)        